In [4]:
class MFInput:
    
    def __init__(self, name, x, y, x0):
        self.name = name
        self.points = [(x[i], y[i]) for i in range(len(x))]
        self.mi = self.getMi(x0)
        
    def getY(self, x1, y1, x2, y2, x0):
        if y1 == y2:
            return y1
        if y1 < y2:
            return (x0 - x1)/(x2 - x1)
        return (x2 - x0)/(x2 - x1)
    
    def getMi(self, x0):
        if x0 < self.points[0][0]:
            return self.points[0][1]
        if x0 > self.points[-1][0]:
            return self.points[-1][1]
        for i in range(1, len(self.points)):
            x1 = self.points[i-1][0]
            x2 = self.points[i][0]
            if x0 >= x1 and x0 <= x2:
                y1 = self.points[i-1][1]
                y2 = self.points[i][1]
                return self.getY(x1,y1,x2,y2,x0)
        return -1

In [2]:
class MFOutput:
    
    def __init__(self, name, x, y):
        self.name = name
        sumX = 0
        nb1 = 0
        self.points = []
        for i in range(len(x)):
            self.points.append((x[i], y[i]))
            if y[i] == 1:
                sumX += x[i]
                nb1 += 1
        self.mi = 0
        self.value = sumX/nb1

In [1]:
from enum import Enum, unique
@unique

class Logic(Enum):
    AND = 1
    OR = 0
    
class Rule:
    
    def __init__(self, mfi1, mfi2, mfo, logic):
        self.mfInput1 = mfi1
        self.mfInput2 = mfi2
        self.mfOutput = mfo
        if logic == Logic.OR:
            self.mfOutput.mi = max(self.mfOutput.mi, max(self.mfInput1.mi, self.mfInput2.mi))
        else:
            self.mfOutput.mi = max(self.mfOutput.mi, min(self.mfInput1.mi, self.mfInput2.mi))

In [5]:
potrosnja = []
potrosnja.append(MFInput("mala", [3, 10], [1, 0], 9))
potrosnja.append(MFInput("srednja", [7, 10, 12, 15], [0, 1, 1, 0], 9))
potrosnja.append(MFInput("velika", [12, 15], [0, 1], 9))

pouzdanost = []
pouzdanost.append(MFInput("visoka", [5, 10], [1, 0], 8))
pouzdanost.append(MFInput("niska", [8, 15], [0, 1], 8))

vrednost = []
vrednost.append(MFOutput("mala", [7, 15], [1, 0]))
vrednost.append(MFOutput("srednja", [7, 15, 25, 40], [0, 1, 1, 0]))
vrednost.append(MFOutput("velika", [25, 40], [0, 1]))

rules = []
rules.append(Rule(potrosnja[0], pouzdanost[0], vrednost[2], Logic.AND))
rules.append(Rule(potrosnja[0], pouzdanost[1], vrednost[1], Logic.AND))
rules.append(Rule(potrosnja[1], pouzdanost[0], vrednost[1], Logic.AND))
rules.append(Rule(potrosnja[1], pouzdanost[1], vrednost[1], Logic.AND))
rules.append(Rule(potrosnja[2], pouzdanost[0], vrednost[1], Logic.AND))
rules.append(Rule(potrosnja[2], pouzdanost[1], vrednost[0], Logic.AND))

numerator = 0
denominator = 0 
for mfo in vrednost:
    numerator += mfo.mi * mfo.value
    denominator += mfo.mi
solution = numerator / denominator

print("Potrosnja:")
for mfi in potrosnja:
    print(mfi.name, mfi.mi)
print("Pouzdanost")
for mfi in pouzdanost:
    print(mfi.name, mfi.mi)
print("Vrednost")
for mfo in vrednost:
    print(mfo.name, mfo.mi, mfo.value)    
print("Resenje je {}".format(solution))

Potrosnja:
mala 0.14285714285714285
srednja 0.6666666666666666
velika 0
Pouzdanost
visoka 0.4
niska 0.0
Vrednost
mala 0 7.0
srednja 0.4 20.0
velika 0.14285714285714285 40.0
Resenje je 25.26315789473684


In [6]:
class Rule:
    def __init__(self, mfo):
        self.mfOutput = mfo
        
    def addInputs(self, mfi1, mfi2, logic):
        self.mfInput1 = mfi1
        self.mfInput2 = mfi2
        if logic == Logic.OR:
            self.mfOutput.mi = max(self.mfOutput.mi, max(self.mfInput1.mi, self.mfInput2.mi))
        else:
            self.mfOutput.mi = max(self.mfOutput.mi, min(self.mfInput1.mi, self.mfInput2.mi))
            
    def addInput(self, mfi):
        self.mfInput = mfi
        self.mfOutput.mi = max(self.mfOutput.mi, self.mfInput.mi)

In [7]:
kvalitet = []
kvalitet.append(MFInput("los", [0, 5], [1, 0], 6.5))
kvalitet.append(MFInput("prosek", [0, 5, 10], [0, 1, 0], 6.5))
kvalitet.append(MFInput("dobar", [5, 10], [0, 1], 6.5))

usluga = []
usluga.append(MFInput("losa", [0, 5], [1, 0], 9.8))
usluga.append(MFInput("prosek", [0, 5, 10], [0, 1, 0], 9.8))
usluga.append(MFInput("dobra", [5, 10], [0, 1], 9.8))

napojnica = []
napojnica.append(MFOutput("mala", [0, 13], [1, 0]))
napojnica.append(MFOutput("srednja", [0, 13, 25], [0, 1, 0]))
napojnica.append(MFOutput("velika", [13, 25], [0, 1]))

rules = []
rules.append(Rule(napojnica[0]).addInputs(kvalitet[0], usluga[0], Logic.OR))
rules.append(Rule(napojnica[1]).addInput(usluga[1]))
rules.append(Rule(napojnica[2]).addInputs(kvalitet[2], usluga[2], Logic.OR))

numerator = 0
denominator = 0 
for mfo in napojnica:
    numerator += mfo.mi * mfo.value
    denominator += mfo.mi
solution = numerator / denominator

print("Kvalitet:")
for mfi in kvalitet:
    print(mfi.name, mfi.mi)
print("Usluga:")
for mfi in usluga:
    print(mfi.name, mfi.mi)
print("Napojnica")
for mfo in napojnica:
    print(mfo.name, mfo.mi, mfo.value)    
print("Resenje je {}".format(solution))

Kvalitet:
los 0
prosek 0.7
dobar 0.3
Usluga:
losa 0
prosek 0.039999999999999855
dobra 0.9600000000000002
Napojnica
mala 0 0.0
srednja 0.039999999999999855 13.0
velika 0.9600000000000002 25.0
Resenje je 24.520000000000003
